<a href="https://colab.research.google.com/github/nourinsilico/qsar/blob/main/QSAR_model_developpment_for_MRSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ChEMBL Data Retrieval and Preprocessing for Methicillin-resistant Staphylococcus aureus**

In [ ]:
! pip install chembl_webresource_client

In [ ]:
!pip install rdkit

In [ ]:
import pandas as pd
import numpy as np
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
target = new_client.target
target_query = target.search('methicillin-resistan staphylococcus aureus ')
targets = pd.DataFrame.from_dict(target_query)
targets

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

In [ ]:
activity = new_client.activity
molecules = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(molecules)

df

In [ ]:
df.to_csv('Methicillin-resistant Staphylococcus aureus_bioactivity_data_raw.csv', index=False)

In [ ]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

In [ ]:
len(df2.canonical_smiles.unique())

In [ ]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

In [ ]:
df3.to_csv('Methicillin-resistant Staphylococcus aureus_bioactivity_data_preprocessed.csv', index=False)

In [ ]:
df4 = pd.read_csv('Methicillin-resistant Staphylococcus aureus_bioactivity_data_preprocessed.csv')

## Bioactivity Classification

In [ ]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [ ]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

In [ ]:
df5.to_csv('Methicillin-resistant Staphylococcus aureus_bioactivity_data_curated.csv', index=False)

In [ ]:
! zip Methicillin-resistant Staphylococcus aureus.zip *.csv

## lipinski rule and Statistical Comparisons of Bioactivity Classes

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
! wget /content/Methicillin-resistant Staphylococcus aureus_bioactivity_data_curated.csv

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Methicillin-resistant Staphylococcus aureus_bioactivity_data_curated.csv')
df

In [ ]:
df_no_smiles = df.drop(columns='canonical_smiles')

In [ ]:
smiles = []

for i in df.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')

In [ ]:
df_clean_smiles = pd.concat([df_no_smiles,smiles], axis=1)
df_clean_smiles

In [ ]:
! pip install numpy

In [ ]:
! pip install rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
import numpy as np

In [ ]:
!pip install rdkit

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)

    return descriptors

In [ ]:
df_lipinski = lipinski(df_clean_smiles.canonical_smiles)
df_lipinski

In [ ]:
df_lipinski

In [ ]:
df

In [ ]:
df_combined = pd.concat([df,df_lipinski], axis=1)

In [ ]:
df_combined

In [ ]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)

    return x

In [ ]:
df_combined.standard_value.describe()

In [ ]:
-np.log10( (10**-9)* 100000000 )

In [ ]:
-np.log10( (10**-9)* 10000000000 )

In [ ]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)

    return x

In [ ]:
df_norm.standard_value_norm.describe()

In [ ]:
df_final = pIC50(df_norm)
df_final

In [ ]:
df_final.pIC50.describe()

In [ ]:
df_final.to_csv('Methicillin-resistant Staphylococcus aureus_bioactivity_data_3class_pIC50.csv')

In [ ]:
df_2class = df_final[df_final['class'] != 'intermediate']
df_2class

In [ ]:
df_2class.to_csv('Methicillin-resistant Staphylococcus aureus_bioactivity_data_2class_pIC50.csv')

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x='class', data=df_2class, edgecolor='black')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.pdf')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'pIC50', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.pdf')

In [ ]:
def mannwhitney(descriptor, verbose=False):

  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'class']
  df = df_2class[selection]
  active = df[df['class'] == 'active']
  active = active[descriptor]

  selection = [descriptor, 'class']
  df = df_2class[selection]
  inactive = df[df['class'] == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'

  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

In [ ]:
mannwhitney('pIC50')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'MW', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')

In [ ]:
mannwhitney('MW')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'LogP', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.pdf')

In [ ]:
mannwhitney('LogP')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'NumHDonors', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.pdf')

In [ ]:
mannwhitney('NumHDonors')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'NumHAcceptors', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcceptors.pdf')

In [ ]:
mannwhitney('NumHAcceptors')

## scaffold

In [ ]:
# Install necessary packages
!pip install pandas
!pip install rdkit-pypi


# Import required libraries
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Scaffolds import MurckoScaffold

from google.colab import files
import matplotlib.pyplot as plt
import os
import zipfile

# Upload the CSV file
uploaded = files.upload()

# Read the CSV file into a pandas DataFrame
filename = list(uploaded.keys())[0]  # Get the uploaded file name
df = pd.read_csv(filename)

# Assuming the ChEMBL IDs are in a column named 'molecule_chembl_id'
# Adjust the column name if your CSV is different
if 'molecule_chembl_id' in df.columns:
    chembl_ids = df['molecule_chembl_id'].dropna().tolist()
    print(f"Extracted {len(chembl_ids)} ChEMBL IDs from the CSV.")
else:
    print("No column named 'molecule_chembl_id' found in the CSV.")
    chembl_ids = []

# Function to fetch SMILES from ChEMBL API
def get_smiles_from_chembl(chembl_id):
    molecule = new_client.molecule
    try:
        res = molecule.get(chembl_id)
        if res and 'molecule_structures' in res and res['molecule_structures']:
            return res['molecule_structures']['canonical_smiles']
        else:
            print(f"Failed to fetch data for {chembl_id}: No molecule structure found")
            return None
    except Exception as e:
        print(f"An error occurred while fetching data for {chembl_id}: {e}")
        return None

# Prepare to store images for original molecules and their scaffolds
original_images = []
scaffold_images = []

# Create a list to store the SMILES strings
smiles_list = []

# Process each ChEMBL ID
for chembl_id in chembl_ids:
    print(f"\nProcessing {chembl_id}...")

    # Fetch SMILES for the compound
    smiles = get_smiles_from_chembl(chembl_id)

    # If SMILES was successfully fetched
    if smiles:
        # Convert the SMILES to an RDKit molecule object
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            print(f"SMILES for {chembl_id}: {smiles}")

            # Draw and store the original molecule
            original_mol_img = Draw.MolToImage(mol)
            original_images.append((chembl_id, original_mol_img))

            # Generate the Murcko scaffold
            scaffold = MurckoScaffold.GetScaffoldForMol(mol)

            # Draw and store the scaffold structure
            scaffold_img = Draw.MolToImage(scaffold)
            scaffold_images.append((chembl_id, scaffold_img))

            # Add the smiles to the smiles_list
            smiles_list.append(smiles)
        else:
            print(f"Invalid SMILES for {chembl_id}.")
    else:
        print(f"Could not retrieve SMILES for {chembl_id}. Please check if the ID is valid.")

# Create a directory to save the scaffold images
os.makedirs('scaffolds', exist_ok=True)

# Generate Murcko scaffolds and save them as images
for idx, smiles in enumerate(smiles_list):
    mol = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    img = Draw.MolToImage(scaffold)
    img.save(f"scaffolds/scaffold_{idx+1}_{chembl_ids[idx]}.png")
# Zip the scaffold images
zip_filename = "scaffold_"


In [ ]:
# Generate Murcko scaffolds and identify unique ones
scaffolds = []
unique_scaffolds = set()
for idx, smiles in enumerate(smiles_list):
    mol = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffolds.append(scaffold)
    unique_scaffolds.add(Chem.MolToSmiles(scaffold))

# Count the number of unique scaffolds
num_unique_scaffolds = len(unique_scaffolds)

print(f"Number of unique scaffolds: {num_unique_scaffolds}")

**PCA**

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# 1. Load your scaffold data:
# Assuming you have a CSV file named 'scaffold_examples.csv'
# with a column named 'Scaffold SMILES'
scaffold_df = pd.read_csv('S2.csv')
smiles_list = scaffold_df['Scaffold SMILES'].tolist()

# 2. Generate fingerprints for each scaffold:
fingerprints = []
for smiles in smiles_list:
    # Check if the SMILES string is valid and not NaN
    if isinstance(smiles, str) and smiles != 'nan' and smiles != '':  # Check if smiles is a valid string and not 'nan'
        mol = Chem.MolFromSmiles(smiles)
        if mol: # Check if mol is not None
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
            fingerprints.append(fp)
        else:
            print(f"Invalid SMILES: {smiles}. Skipping.") # Print invalid SMILES for debugging
    else:
        print(f"Invalid or missing SMILES: {smiles}. Skipping.") # Print invalid or missing SMILES for debugging

# 3. Convert fingerprints to a NumPy array:
X = np.array(fingerprints, dtype=np.int32)

# 4. Standardize the data:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. Perform PCA:
pca = PCA(n_components=2)  # Choose the number of components
principalComponents = pca.fit_transform(X_scaled)

# 6. Create a DataFrame for the principal components:
principalDf = pd.DataFrame(data=principalComponents,
                           columns=['principal component 1', 'principal component 2'])

# 7. Visualize the results:
plt.figure(figsize=(8, 6))
plt.scatter(principalDf['principal component 1'],
            principalDf['principal component 2'], alpha=0.5)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Scaffolds')
plt.show()

## Distribution of pIC50 Values

In [ ]:
filename = "/content/S2 Methicillin resistant Staphylococcus aureus bioactivity data_2class_pIC50.csv"
df = pd.read_csv(filename,sep=" ",names=["SMILES","Name","pIC50"])

In [ ]:
!pip install pandas matplotlib
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('/content/S2 Methicillin resistant Staphylococcus aureus bioactivity data_3class_pIC50.csv')

# Analyze the distribution of pIC50 values
plt.hist(df['pIC50'].dropna(), bins=20)
plt.xlabel('pIC50')
plt.ylabel('Frequency')
plt.title('Distribution of pIC50 Values')
plt.show()

# Calculate basic statistics
print(df['pIC50'].describe())

## molecular descriptors

In [ ]:
!pip install rdkit pandas datamol molfeat numpy scikit-learn yellowbrick

In [ ]:
import pandas as pd
import datamol as dm
from molfeat.calc import FPCalculator
from molfeat.trans import MoleculeTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from yellowbrick.regressor import prediction_error, residuals_plot

In [ ]:
calc = FPCalculator("maccs")
calc = FPCalculator("avalon")
calc = FPCalculator("ecfp")
calc = FPCalculator("topological")
calc = FPCalculator("atompair")
calc = FPCalculator("rdkit")
calc = FPCalculator("pattern")
calc = FPCalculator("layered")
calc = FPCalculator("map4")
calc = FPCalculator("erg")


In [ ]:
trans = MoleculeTransformer(calc)

In [ ]:
df = pd.read_csv('/content/S2 Methicillin resistant Staphylococcus aureus bioactivity data_2class_pIC50.csv')

In [ ]:
%%time
with dm.without_rdkit_log():
    df['fp'] = trans.transform(df.canonical_smiles.values)

## train_test_split

In [ ]:
train, test = train_test_split(df)

In [ ]:
from sklearn.model_selection import train_test_split
train_size = 0.8
train, test = train_test_split(df, train_size=train_size, random_state=42)

## model

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
model = XGBRegressor()

## internal and external validation

## R2

In [ ]:
%%time
visualizer = prediction_error(model,np.stack(train.fp),train.pIC50)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
y_test = test.pIC50

In [ ]:
# Fit the model to your training data
model.fit(np.stack(train.fp), train.pIC50)

# Now you can predict on your test data
y_pred = model.predict(np.stack(test.fp))

In [ ]:
y_pred = model.predict(np.stack(test.fp))

## Q2

In [ ]:
q2 = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - y_test.mean()) ** 2))
print(f"Q^2 on the test set: {q2}")

## Q2F3

In [ ]:
def q2f3(y_true, y_pred, num_features): # Add num_features as an argument
  """
  Calculates the Q^2F3 metric for model performance.

  Args:
    y_true: Array-like, true values.
    y_pred: Array-like, predicted values.
    num_features: The number of features used to train the model.

  Returns:
    float: Q^2F3 value.
  """
  ss_res = np.sum((y_true - y_pred)**2)
  ss_tot = np.sum((y_true - np.mean(y_true))**2)
  r2 = 1 - (ss_res / ss_tot)

  # For tree-based models, you might consider using a different
  # metric or adapting the formula. For example, you could
  # consider the number of features instead of coefficients.
  # Here's an example of how you might adapt the formula:

  q2f3 = 1 - (ss_res / ss_tot) * ((len(y_true) - 1) / (len(y_true) - num_features - 1)) # Use num_features

  return q2f3

# Assuming you have y_test and y_pred defined (as in your provided code)
# Replace 9 with the actual number of features used in your model
q2f3_value = q2f3(y_test, y_pred, 9)
print(f"Q^2F3 on the test set: {q2f3_value:.3f}")

## CCC

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def ccc(y_true, y_pred):
  """
  Calculates Lin's Concordance Correlation Coefficient (CCC).

  Args:
    y_true: Array-like, true values.
    y_pred: Array-like, predicted values.

  Returns:
    float: CCC value.
  """
  mean_true = np.mean(y_true)
  mean_pred = np.mean(y_pred)

  var_true = np.var(y_true)
  var_pred = np.var(y_pred)

  cov = np.cov(y_true, y_pred, bias=True)[0][1]

  ccc = (2 * cov) / (var_true + var_pred + (mean_true - mean_pred) ** 2)

  return ccc

# Assuming you have y_test and y_pred defined
ccc_value = ccc(y_test, y_pred)
print(f"CCC on the test set: {ccc_value:.3f}")

## Q2LMO

In [ ]:
from sklearn.model_selection import KFold

def q2lmo(model, X, y, k=5):
  """
  Calculates the Leave-Many-Out (LMO) Q^2 for a model using k-fold CV.

  Args:
    model: A scikit-learn compatible model object.
    X: Array-like, feature matrix.
    y: Array-like, target variable.
    k: The number of folds to use in cross-validation.

  Returns:
    float: Q^2LMO value.
  """
  kf = KFold(n_splits=k)
  y_pred = np.zeros_like(y)

  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    y_pred[test_index] = model.predict(X_test)

  ss_res = np.sum((y - y_pred)**2)
  ss_tot = np.sum((y - np.mean(y))**2)
  q2 = 1 - (ss_res / ss_tot)

  return q2

## Y-scrambling

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Specify features (X) and target (y)
# Exclude the 'molecule_chembl_id' column, which contains strings
X = df[['pIC50']].values  # Select only numerical features (descriptors)
y = df.pIC50.values   # Assuming the target property is the 'pIC50' column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model on the original data
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
r2_original = r2_score(y_test, y_pred)
rmse_original = np.sqrt(mean_squared_error(y_test, y_pred))


# Y-scrambling
n_iterations = 100  # Number of random permutations
r2_scrambled = []

for i in range(n_iterations):
    np.random.shuffle(y_train)  # Shuffle the target values
    model.fit(X_train, y_train)  # Train the model with scrambled y
    y_pred_scrambled = model.predict(X_test)
    r2_scrambled.append(r2_score(y_test, y_pred_scrambled))

# Plot the results
plt.figure(figsize=(8, 6))
plt.hist(r2_scrambled, bins=15, color='skyblue', edgecolor='black', alpha=0.7, label='Scrambled R^2')
plt.axvline(r2_original, color='red', linestyle='dashed', linewidth=2, label=f'Original R^2 ({r2_original:.3f})')
plt.xlabel('R^2 Values')
plt.ylabel('Frequency')
plt.title('Y-Scrambling Analysis')
plt.legend()
plt.show()

# Statistical summary
print("\nY-Scrambling Results:")
print(f"Mean R^2 (Scrambled): {np.mean(r2_scrambled):.3f}")
print(f"Standard Deviation of R^2 (Scrambled): {np.std(r2_scrambled):.3f}")

## William plot

In [ ]:
# Calculate leverage for training and test sets
H_train = np.dot(X_train, np.linalg.inv(np.dot(X_train.T, X_train))).dot(X_train.T)
H_test = np.dot(X_test, np.linalg.inv(np.dot(X_train.T, X_train))).dot(X_train.T)

leverage_train = np.diag(H_train)
leverage_test = np.diag(H_test)

# Leverage threshold h*
n_train, p = X_train.shape
h_star = (3 * (p + 1)) / n_train

# Plotting the Williams Plot (Leverage vs Standardized Residuals)
plt.figure(figsize=(10, 6))

# Training data plot
#plt.scatter(leverage_train, train_residuals, color='blue', label='Training set')

# Test data plot
plt.scatter(leverage_test, test_residuals, color='red', label='Test set')

# Plot horizontal lines for residual thresholds (±3 standard deviations)
plt.axhline(y=3, color='gray', linestyle='--', label='±3 Residual Threshold')
plt.axhline(y=-3, color='gray', linestyle='--')

# Plot vertical line for leverage threshold h*
plt.axvline(x=h_star, color='green', linestyle='--', label=f'h* = {h_star:.3f}')

# Plot formatting
plt.xlabel('Leverage (h)')
plt.ylabel('Standardized Residuals')
plt.title('Williams Plot (Applicability Domain)')
plt.legend()
plt.grid(True)

# Show plot
plt.show()

# Optionally save plot
plt.savefig('williams_plot.png', dpi=300)